# Ricavi

##### setup

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


## 1) Scostamento costo totale Ricavi (Vendite)

In [ ]:
dfVendite.rename(columns = {'budget/cons':'budget'}, inplace = True)
dfVendite.loc[dfVendite.loc[:, "budget"] ==
              "Consuntivo", "budget"] = "CONSUNTIVO"
dfVendite.loc[dfVendite.loc[:, "budget"]
              == "Budget", "budget"] = "BUDGET"


#Joining with the exchange rate 
dfVendite = sqldf("SELECT * FROM  dfClienti C JOIN dfVendite V ON C.Nr = V.NrOrigine")

#La prima colonna è il codice del cliente

dfVendite = sqldf("SELECT * FROM dfVendite V JOIN dfCambio C ON  V.Valuta = C.CodiceValuta AND V.budget = C.Anno ")
del dfVendite['Anno']
del dfVendite['CodiceValuta']
del dfVendite['NrOrigine']

In [ ]:
VenditeBudget = sqldf("SELECT * FROM dfVendite WHERE budget = 'BUDGET' or budget = 'Budget'")
VenditeConsuntivo = sqldf("SELECT * FROM dfVendite WHERE budget = 'CONSUNTIVO' or budget = 'Consuntivo'")

In [ ]:
#Calcolo per il budget

calcoloBudget = sqldf("SELECT * ,SUM(ImportoVenditaValutaLocaleTOTALEVENDITA*TassoCambioMedio) AS VenditeB FROM VenditeBudget GROUP BY Nr")

#Calcolo per il consuntivo
calcoloConsuntivo = sqldf("SELECT *, SUM(ImportoVenditaValutaLocaleTOTALEVENDITA*TassoCambioMedio) AS VenditeC FROM VenditeConsuntivo GROUP BY Nr")

scostamentoVenditeTotale = sqldf(
    "SELECT B.Nr, VenditeB, VenditeC, VenditeB-VenditeC AS Differenza FROM calcoloBudget AS B JOIN calcoloConsuntivo AS C ON B.Nr = C.Nr ")
sumScostamentoVenditeTotale = sqldf(
    "SELECT SUM(Differenza) FROM scostamentoVenditeTotale")

print('Scostamento totale delle vendite: '+ str(sumScostamentoVenditeTotale))
scostamentoVenditeTotale.head()


## 2) scostamenti vendinte per NrArticolo

In [ ]:
# quantità totale di codici MP 
countVenditec = sqldf('select distinct NrArticolo, count(*) from venditeConsuntivo group by NrArticolo order by count(*) desc')
countVenditeb = sqldf('select distinct NrArticolo, count(*) from venditeBudget group by NrArticolo order by count(*) desc')
print("Consuntivo")
print(countVenditec)
print("Budget")
print(countVenditeb)


In [ ]:
venditeConsuntivoByArt = sqldf('select NrArticolo, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as sumVenditaTotale from venditeConsuntivo group by NrArticolo order by sumVenditaTotale desc')
venditeBudgetByArt = sqldf('select NrArticolo, sum(ImportoVenditaValutaLocaleTOTALEVENDITA) as sumVenditaTotale from venditeBudget group by NrArticolo order by sumVenditaTotale desc')


In [ ]:
scostamentoVenditeArt = sqldf('''select c.NrArticolo, b.sumVenditaTotale as sumVenditaTotaleBudget, c.sumVenditaTotale as sumVenditaTotaleConsuntivo, b.sumVenditaTotale-c.sumVenditaTotale as scostamento 
from venditeConsuntivoByArt as c join venditeBudgetByArt as b 
on c.NrArticolo = b.NrArticolo
order by scostamento desc''')

scostamentoVenditeArt

,NrArticolo,sumVenditaTotaleBudget,sumVenditaTotaleConsuntivo,scostamento
0,ART0000762,35334.60,7160.00,28174.60
1,ART0003780,6028.80,1312.00,4716.80
2,ART0000812,6429.60,2280.00,4149.60
3,ART0003610,4560.00,600.00,3960.00
4,ART0000826,4371.00,930.00,3441.00
...,...,...,...,...
408,ART0000811,7426.00,22120.00,-14694.00
409,ART0004889,450.00,16200.00,-15750.00
410,ART0005136,17920.21,34340.42,-16420.21
411,ART0000847,1776.38,21330.00,-19553.62


In [ ]:
sqldf('select sum (scostamento) from scostamentoVenditeArt') 

,sum (scostamento)
0,-198274.52
